In [3]:
from policyengine_us import Simulation
from policyengine_core.reforms import Reform
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from policyengine_core.charts import format_fig

In [4]:
reform = Reform.from_dict(
    {
        "gov.contrib.aca.ptc_additional_bracket.in_effect": {
            "2026-01-01.2100-12-31": True
        }
    },
    country_id="us",
)

reform2 = Reform.from_dict(
    {
        "gov.contrib.aca.ptc_simplified_bracket.in_effect": {
            "2026-01-01.2100-12-31": True
        }
    },
    country_id="us",
)

reform3 = Reform.from_dict({
  "gov.aca.required_contribution_percentage[0].amount": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.aca.required_contribution_percentage[1].amount": {
    "2025-01-01.2100-12-31": 0
  },
  "gov.aca.required_contribution_percentage[2].amount": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.aca.required_contribution_percentage[3].amount": {
    "2026-01-01.2100-12-31": 0.02
  },
  "gov.aca.required_contribution_percentage[4].amount": {
    "2026-01-01.2100-12-31": 0.04
  },
  "gov.aca.required_contribution_percentage[5].amount": {
    "2026-01-01.2100-12-31": 0.06
  },
  "gov.aca.required_contribution_percentage[6].amount": {
    "2026-01-01.2100-12-31": 0.085
  },
  "gov.aca.ptc_income_eligibility[2].amount": {
    "2026-01-01.2100-12-31": True
  }
}, country_id="us")

In [5]:
from plotly.subplots import make_subplots

In [6]:
import plotly.graph_objects as go

In [7]:
import numpy as np

In [8]:
from policyengine_core.reforms import Reform

In [9]:
reform = Reform.from_dict(
    {
        "gov.contrib.aca.ptc_additional_bracket.in_effect": {
            "2026-01-01.2100-12-31": True
        }
    },
    country_id="us",
)

reform2 = Reform.from_dict(
    {
        "gov.contrib.aca.ptc_simplified_bracket.in_effect": {
            "2026-01-01.2100-12-31": True
        }
    },
    country_id="us",
)

reform3 = Reform.from_dict({
  "gov.aca.required_contribution_percentage[0].amount": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.aca.required_contribution_percentage[1].amount": {
    "2025-01-01.2100-12-31": 0
  },
  "gov.aca.required_contribution_percentage[2].amount": {
    "2026-01-01.2100-12-31": 0
  },
  "gov.aca.required_contribution_percentage[3].amount": {
    "2026-01-01.2100-12-31": 0.02
  },
  "gov.aca.required_contribution_percentage[4].amount": {
    "2026-01-01.2100-12-31": 0.04
  },
  "gov.aca.required_contribution_percentage[5].amount": {
    "2026-01-01.2100-12-31": 0.06
  },
  "gov.aca.required_contribution_percentage[6].amount": {
    "2026-01-01.2100-12-31": 0.085
  },
  "gov.aca.ptc_income_eligibility[2].amount": {
    "2026-01-01.2100-12-31": True
  }
}, country_id="us")

In [10]:
situation_florida = {
  "people": {
    "parent1": {
      "age": {
        "2026": 40
      }
    },
    "parent2": {
      "age": {
        "2026": 40
      }
    },
    "child1": {
      "age": {
        "2026": 10
      }
    },
    "child2": {
      "age": {
        "2026": 8
      }
    }
  },
  "families": {
    "your family": {
      "members": [
        "parent1",
        "parent2",
        "child1",
        "child2"
      ]
    }
  },
  "spm_units": {
    "your household": {
      "members": [
        "parent1",
        "parent2",
        "child1",
        "child2"
      ]
    }
  },
  "tax_units": {
    "your tax unit": {
      "members": [
        "parent1",
        "parent2",
        "child1",
        "child2"
      ]
    }
  },
  "households": {
    "your household": {
      "members": [
        "parent1",
        "parent2",
        "child1",
        "child2"
      ],
      "state_name": {
        "2026": "FL"
      },
      "county_fips": {
        "2026": "12057"
      }
    }
  },
  "marital_units": {
    "parents marital unit": {
      "members": [
        "parent1",
        "parent2"
      ]
    }
  },
  "axes": [
    [
      {
        "name": "employment_income",
        "count": 800,
        "min": 0,
        "max": 120000
      }
    ]
  ]
}

In [11]:
simulation_florida = Simulation(
    situation=situation_florida,
)
reformed_simulation_florida = Simulation(
    situation=situation_florida,
    reform=reform,
)
reformed_simulation_florida2 = Simulation(
    situation=situation_florida,
    reform=reform2,
)
reformed_simulation_florida3 = Simulation(
    situation=situation_florida,
    reform=reform3,
)

In [12]:
import copy
import pandas as pd
from policyengine_us import Simulation

PERIOD = 2026

# ------------------------------------------------------------------
# Helper: get the tax unit's 2026 FPG from the situation
# ------------------------------------------------------------------
def get_tax_unit_fpg(base_situation: dict, period=PERIOD) -> float:
    """Return the tax unit FPG for the given situation/year (first tax unit)."""
    sit = copy.deepcopy(base_situation)
    sit.pop("axes", None)

    # Ensure income isn't interfering (FPG shouldn't depend on income, but be safe)
    for person in sit["people"].values():
        person.setdefault("employment_income", {})
        person["employment_income"][str(period)] = 0

    sim = Simulation(situation=sit)
    fpg = sim.calculate("tax_unit_fpg", map_to="tax_unit", period=period)[0]
    return float(fpg)

# ------------------------------------------------------------------
# 1) Convenience: run a one-income Florida family simulation
# ------------------------------------------------------------------
def calc_ptc_for_income(base_situation: dict, income: float, *, reform_to_use=None, period=PERIOD):
    """
    Return ACA PTC (household-level) for the given income and year.
    """
    sit = copy.deepcopy(base_situation)
    sit.pop("axes", None)

    # Assign all income to the first parent
    sit["people"]["parent1"]["employment_income"] = {str(period): income}

    sim = Simulation(situation=sit, reform=reform_to_use)
    return sim.calculate("aca_ptc", map_to="household", period=period)[0]

# ------------------------------------------------------------------
# 2) Build targets from model-derived FPG and compute PTCs
# ------------------------------------------------------------------
fpg_2026 = get_tax_unit_fpg(situation_florida, period=PERIOD)

percent_targets = {
    "138 % FPL": 1.38,
    "300 % FPL": 3.00,
    "400 % FPL": 4.00,
}

rows = []
for label, mult in percent_targets.items():
    inc = round(fpg_2026 * mult, 2)
    rows.append({
        "income_label": f"{label} (${inc:,.2f})",
        "income_usd": inc,
        "ptc_baseline":   calc_ptc_for_income(situation_florida, inc, reform_to_use=None, period=PERIOD),
        "ptc_stepped_proposal": calc_ptc_for_income(situation_florida, inc, reform_to_use=reform, period=PERIOD),
        "ptc_linear_proposal": calc_ptc_for_income(situation_florida, inc, reform_to_use=reform2, period=PERIOD),
        "ptc_ira_extension": calc_ptc_for_income(situation_florida, inc, reform_to_use=reform3, period=PERIOD),
    })

ptc_df = pd.DataFrame(rows)
ptc_df

,income_label,income_usd,ptc_baseline,ptc_stepped_proposal,ptc_linear_proposal,ptc_ira_extension
0,"138 % FPL ($45,446.42)",45446.42,10588.647461,12240.223633,11494.902344,12240.223633
1,"300 % FPL ($98,796.57)",98796.57,9726.927734,13362.642578,11386.710938,13466.378906
2,"400 % FPL ($131,728.77)",131728.77,0.000000,5919.965820,3285.390625,8370.121094


In [13]:
# Get household-level values for Florida
household_income_florida = simulation_florida.calculate("employment_income", map_to="household", period=2026)
baseline_florida_per_capita_chip = simulation_florida.calculate("per_capita_chip", map_to="household", period=2026)
baseline_florida_aca_ptc = simulation_florida.calculate("aca_ptc", map_to="household", period=2026)
baseline_florida_medicaid_cost = simulation_florida.calculate("medicaid_cost", map_to="household", period=2026)
baseline_florida_net_income_including_health_benefits = simulation_florida.calculate("household_net_income_including_health_benefits", map_to="household", period=2026)
baseline_florida_slcsp = simulation_florida.calculate("slcsp", map_to="household", period=2026)

reform_florida_per_capita_chip = reformed_simulation_florida.calculate("per_capita_chip", map_to="household", period=2026)
reform_florida_aca_ptc = reformed_simulation_florida.calculate("aca_ptc", map_to="household", period=2026)
reform_florida_medicaid_cost = reformed_simulation_florida.calculate("medicaid_cost", map_to="household", period=2026)
reform_florida_net_income_including_health_benefits = reformed_simulation_florida.calculate("household_net_income_including_health_benefits", map_to="household", period=2026)

reform2_florida_per_capita_chip = reformed_simulation_florida2.calculate("per_capita_chip", map_to="household", period=2026)
reform2_florida_aca_ptc = reformed_simulation_florida2.calculate("aca_ptc", map_to="household", period=2026)
reform2_florida_medicaid_cost = reformed_simulation_florida2.calculate("medicaid_cost", map_to="household", period=2026)
reform2_florida_net_income_including_health_benefits = reformed_simulation_florida2.calculate("household_net_income_including_health_benefits", map_to="household", period=2026)

reform3_florida_per_capita_chip = reformed_simulation_florida3.calculate("per_capita_chip", map_to="household", period=2026)
reform3_florida_aca_ptc = reformed_simulation_florida3.calculate("aca_ptc", map_to="household", period=2026)
reform3_florida_medicaid_cost = reformed_simulation_florida3.calculate("medicaid_cost", map_to="household", period=2026)
reform3_florida_net_income_including_health_benefits = reformed_simulation_florida3.calculate("household_net_income_including_health_benefits", map_to="household", period=2026)

# Calculate total benefits for each scenario
baseline_florida_total = [sum(x) for x in zip(baseline_florida_per_capita_chip, baseline_florida_aca_ptc, baseline_florida_medicaid_cost)]
reform_florida_total = [sum(x) for x in zip(reform_florida_per_capita_chip, reform_florida_aca_ptc, reform_florida_medicaid_cost)]
reform2_florida_total = [sum(x) for x in zip(reform2_florida_per_capita_chip, reform2_florida_aca_ptc, reform2_florida_medicaid_cost)]
reform3_florida_total = [sum(x) for x in zip(reform3_florida_per_capita_chip, reform3_florida_aca_ptc, reform3_florida_medicaid_cost)]

In [14]:
GRAY = "#808080"
BLUE_PRIMARY = "#2C6496"
TEAL_ACCENT = "#39C6C0"
DARK_GRAY = "#616161"
PURPLE = "#9467BD"

In [18]:
# Create Florida graph
fig_florida = go.Figure()

# Add baseline traces (solid lines)
fig_florida.add_trace(go.Scatter(
    x=household_income_florida, 
    y=baseline_florida_aca_ptc, 
    mode='lines', 
    name='ACA PTC (Baseline)', 
    line=dict(color=GRAY, width=2)
))

# Add reform traces
fig_florida.add_trace(go.Scatter(
    x=household_income_florida, 
    y=reform_florida_aca_ptc, 
    mode='lines', 
    name='ACA PTC (Stepped Proposal)', 
    line=dict(color=BLUE_PRIMARY, width=2)
))

fig_florida.add_trace(go.Scatter(
    x=household_income_florida, 
    y=reform2_florida_aca_ptc, 
    mode='lines', 
    name='ACA PTC (Linear Proposal)', 
    line=dict(color=TEAL_ACCENT, width=2)
))

fig_florida.add_trace(go.Scatter(
    x=household_income_florida, 
    y=reform3_florida_aca_ptc, 
    mode='lines', 
    name='ACA PTC (IRA Extension)', 
    line=dict(color=PURPLE, width=2)
))


# Update layout
fig_florida.update_layout(
    title='Florida Household (Two parents with two children) - Program Benefits by Income Level',
    xaxis_title='Household Income',
    yaxis_title='Benefit Amount',
    legend_title='Programs',
    xaxis=dict(tickformat='$,.0f', range=[0, 120000]),
    yaxis=dict(tickformat='$,.0f'),
    height=600,
    width=1000
)

fig_florida = format_fig(fig_florida)
fig_florida.show()

In [19]:
#Household net income graphs
import plotly.graph_objects as go

# ---------- Florida family of 4 ----------
fig_fl = go.Figure()

# Baseline (solid)
fig_fl.add_trace(go.Scatter(
    x=household_income_florida,
    y=baseline_florida_net_income_including_health_benefits,
    mode='lines',
    name='Baseline',
    line=dict(color=GRAY, width=2)
))

# Reform 1 (stepped)
fig_fl.add_trace(go.Scatter(
    x=household_income_florida,
    y=reform_florida_net_income_including_health_benefits,
    mode='lines',
    name='Stepped Proposal',
    line=dict(color=BLUE_PRIMARY, width=2)
))

# Reform 2 (linear)
fig_fl.add_trace(go.Scatter(
    x=household_income_florida,
    y=reform2_florida_net_income_including_health_benefits,
    mode='lines',
    name='Linear Proposal',
    line=dict(color=TEAL_ACCENT, width=2)
))

# Reform 3 (IRA extension)
fig_fl.add_trace(go.Scatter(
    x=household_income_florida,
    y=reform3_florida_net_income_including_health_benefits,
    mode='lines',
    name='IRA Extension',
    line=dict(color=PURPLE, width=2)
))

# Layout
fig_fl.update_layout(
    title='Florida Household (Two parents with two children) – Health-Adjusted Net Income by Household Income',
    xaxis_title='Household Income',
    yaxis_title='Health-Adjusted Net Income',
    legend_title='Scenario',
    xaxis=dict(tickformat='$,.0f', range=[0, 120_000]),
    yaxis=dict(tickformat='$,.0f'),
    height=600,
    width=1000
)

fig_fl = format_fig(fig_fl)
fig_fl.show()

# --- Δ Health-adjusted net income (Reform – Baseline) ---
delta_fl = (
    reform_florida_net_income_including_health_benefits
    - baseline_florida_net_income_including_health_benefits
)

delta_fl2 = (
    reform2_florida_net_income_including_health_benefits
    - baseline_florida_net_income_including_health_benefits
)

delta_fl3 = (
    reform3_florida_net_income_including_health_benefits
    - baseline_florida_net_income_including_health_benefits
)

fig_delta_fl = go.Figure()

fig_delta_fl.add_trace(go.Scatter(
    x=household_income_florida,
    y=delta_fl,
    mode='lines',
    name='Stepped Proposal',
    line=dict(color=BLUE_PRIMARY, width=2)
))

fig_delta_fl.add_trace(go.Scatter(
    x=household_income_florida,
    y=delta_fl2,
    mode='lines',
    name='Linear Proposal',
    line=dict(color=TEAL_ACCENT, width=2)
))

fig_delta_fl.add_trace(go.Scatter(
    x=household_income_florida,
    y=delta_fl3,
    mode='lines',
    name='IRA Extension',
    line=dict(color=PURPLE, width=2)
))

fig_delta_fl.update_layout(
    title='Florida Household (Two parents with two children) – Impact of Premium Tax Credit Proposals',
    xaxis_title='Household Income',
    yaxis_title='Δ Net Income',
    xaxis=dict(tickformat='$,.0f', range=[0, 120_000]),
    yaxis=dict(tickformat='$,.0f', zeroline=True, zerolinewidth=1),
    height=600,
    width=1000,
    legend=dict(orientation='h')
)

fig_delta_fl = format_fig(fig_delta_fl)
fig_delta_fl.show()

In [20]:
# ========================================================================
# MTR CALCULATION FOR FLORIDA HOUSEHOLD
# Uses axes-based simulation for net income, then calculates MTR manually
# ========================================================================

# Step 1: Create situation with axes for vectorized net income calculation
situation_fl_for_mtr = {
    "people": {
        "parent1": {
            "age": {"2026": 40},
            "employment_income": {"2026": 0},
            "self_employment_income": {"2026": 0}
        },
        "parent2": {
            "age": {"2026": 40},
            "employment_income": {"2026": 0},
            "self_employment_income": {"2026": 0}
        },
        "child1": {
            "age": {"2026": 10},
            "employment_income": {"2026": 0},
            "self_employment_income": {"2026": 0}
        },
        "child2": {
            "age": {"2026": 8},
            "employment_income": {"2026": 0},
            "self_employment_income": {"2026": 0}
        }
    },
    "families": {
        "your family": {
            "members": ["parent1", "parent2", "child1", "child2"]
        }
    },
    "spm_units": {
        "your household": {
            "members": ["parent1", "parent2", "child1", "child2"]
        }
    },
    "tax_units": {
        "your tax unit": {
            "members": ["parent1", "parent2", "child1", "child2"]
        }
    },
    "households": {
        "your household": {
            "members": ["parent1", "parent2", "child1", "child2"],
            "state_name": {"2026": "FL"},
            "county_fips": {"2026": "12057"}
            
        }
    },
    "marital_units": {
        "parents marital unit": {
            "members": ["parent1", "parent2"]
        }
    },
    "axes": [[
        {
            "name": "employment_income",
            "min": 0,
            "max": 120_000,
            "count": 200,
            "period": "2026"
        }
    ]]
}

# Step 2: Calculate baseline net income using axes
sim_fl_baseline = Simulation(situation=situation_fl_for_mtr)
household_income_fl_mtr = sim_fl_baseline.calculate("employment_income", map_to="household", period=2026)
baseline_fl_net_income = sim_fl_baseline.calculate(
    "household_net_income_including_health_benefits",
    map_to="household",
    period=2026
)

# Step 3: Calculate reform net income using axes
sim_fl_reform = Simulation(situation=situation_fl_for_mtr, reform=reform)
reform_fl_net_income = sim_fl_reform.calculate(
    "household_net_income_including_health_benefits",
    map_to="household",
    period=2026
)

# Step 3b: Calculate reform2 net income using axes
sim_fl_reform2 = Simulation(situation=situation_fl_for_mtr, reform=reform2)
reform2_fl_net_income = sim_fl_reform2.calculate(
    "household_net_income_including_health_benefits",
    map_to="household",
    period=2026
)

# Step 3c: Calculate reform3 net income using axes
sim_fl_reform3 = Simulation(situation=situation_fl_for_mtr, reform=reform3)
reform3_fl_net_income = sim_fl_reform3.calculate(
    "household_net_income_including_health_benefits",
    map_to="household",
    period=2026
)

# Step 4: Calculate MTR from adjacent points
def calc_mtr(incomes, net_incomes):
    """Calculate MTR between adjacent income points."""
    mtrs = []
    mtr_incomes = []
    for i in range(len(incomes) - 1):
        income_change = incomes[i + 1] - incomes[i]
        net_change = net_incomes[i + 1] - net_incomes[i]
        if income_change > 0 and not np.isnan(net_incomes[i]) and not np.isnan(net_incomes[i + 1]):
            mtr = 1 - (net_change / income_change)
            mtrs.append(mtr)
            mtr_incomes.append((incomes[i] + incomes[i + 1]) / 2)
    return np.array(mtr_incomes), np.array(mtrs)

baseline_fl_mtr_incomes, baseline_fl_mtrs = calc_mtr(household_income_fl_mtr, baseline_fl_net_income)
reform_fl_mtr_incomes, reform_fl_mtrs = calc_mtr(household_income_fl_mtr, reform_fl_net_income)
reform2_fl_mtr_incomes, reform2_fl_mtrs = calc_mtr(household_income_fl_mtr, reform2_fl_net_income)
reform3_fl_mtr_incomes, reform3_fl_mtrs = calc_mtr(household_income_fl_mtr, reform3_fl_net_income)

# Step 5: Create the chart
fig_florida_mtr = go.Figure()

fig_florida_mtr.add_trace(go.Scatter(
    x=baseline_fl_mtr_incomes,
    y=np.clip(baseline_fl_mtrs, -1.0, 1.0),
    mode='lines',
    name='Baseline',
    line=dict(color=GRAY, width=2),
    hovertemplate='Income: $%{x:,.0f}<br>Baseline MTR: %{y:.1%}<extra></extra>'
))

fig_florida_mtr.add_trace(go.Scatter(
    x=reform_fl_mtr_incomes,
    y=np.clip(reform_fl_mtrs, -1.0, 1.0),
    mode='lines',
    name='Stepped Proposal',
    line=dict(color=BLUE_PRIMARY, width=2),
    hovertemplate='Income: $%{x:,.0f}<br>Stepped MTR: %{y:.1%}<extra></extra>'
))

fig_florida_mtr.add_trace(go.Scatter(
    x=reform2_fl_mtr_incomes,
    y=np.clip(reform2_fl_mtrs, -1.0, 1.0),
    mode='lines',
    name='Linear Proposal',
    line=dict(color=TEAL_ACCENT, width=2),
    hovertemplate='Income: $%{x:,.0f}<br>Linear MTR: %{y:.1%}<extra></extra>'
))

fig_florida_mtr.add_trace(go.Scatter(
    x=reform3_fl_mtr_incomes,
    y=np.clip(reform3_fl_mtrs, -1.0, 1.0),
    mode='lines',
    name='IRA Extension',
    line=dict(color=PURPLE, width=2),
    hovertemplate='Income: $%{x:,.0f}<br>IRA MTR: %{y:.1%}<extra></extra>'
))

fig_florida_mtr.update_layout(
    title='Marginal tax rate including health benefits - Florida family of 4',
    xaxis_title='Employment income',
    yaxis_title='Marginal tax rate',
    xaxis=dict(
        tickformat='$,.0f',
        range=[0, 120_000],
        gridcolor='lightgray',
        showgrid=True
    ),
    yaxis=dict(
        tickformat='.0%',
        range=[-1.0, 1.0],
        gridcolor='lightgray',
        showgrid=True,
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='gray'
    ),
    height=600,
    width=1000,
    hovermode='x unified',
    plot_bgcolor='white',
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig_florida_mtr = format_fig(fig_florida_mtr)
fig_florida_mtr.show()